# Homework 2：

⾃动编码器模型（AE）；变分⾃动编码器模型（VAE）；条件变分⾃动编码器模型（Conditional VAE）

并将模型应⽤到MNIST 数据上对⼿写体数字进⾏降维、聚类、⽣成和指定⽣成⼯作。

## 1. FC layer实现AE、VAE、CVAE & 画图展示

### 1.1 实现AE模型
 
#### 1.1.1 通用部件Encoder & Decoder的实现


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Encoder实现
# 留了个参数：is_dist
# TODO：判断我们的encoder是输出一个code(AE) or 输出隐变量服从的分布的均值与方差(VAE,CVAE).



class Encoder(nn.Module):
    def __init__(self, x_dim, hidden_size, latent_size, arch="AE", **kwargs) -> None:
        super(Encoder, self).__init__()
        self.mu = nn.Sequential(nn.Linear(x_dim, hidden_size), nn.ReLU(), nn.Linear(hidden_size, latent_size),)
        self.arch = arch
        if self.arch != "AE":  # 若encoder返回的是均值与标准差(VAE、CVAE), 需要额外生成mu和sigma（两架构可以一样）
            self.sigma = nn.Sequential(nn.Linear(x_dim, hidden_size), nn.ReLU(), nn.Linear(hidden_size, latent_size),)

    def forward(self, xs):
        # 实现编码器的forward过程，arch架构的不同取值意味着我们需要不同输出的encoder
        if self.arch == "AE":
            output = self.mu(xs) # AE的输出，只有一个code
        elif self.arch == "VAE":# VAE的输出，有mu和sigma两个输出
            mu = self.mu(xs)
            sigma = self.sigma(xs)
            output = tuple(mu,sigma)
        else: # CVAE的输出，有mu和sigma两个，但是输入还有希望生成数据的label！需要concatenate起来

            


        return output

In [ ]:

# Decoder的实现：不同的自编码器可能需要不同的输入, 借助**otherinputs来处理.
# Review：**收集所有未匹配的关键字参数组成一个dict对象，局部变量kwargs指向此dict对象
# 对于AE的情况, 只需要zs作为输入即可,
# 对于VAE,CVAE的情况, 我们可能需要code服从分布的均值与方差(对于CVAE, 还需要类别的指示变量)

# 在实现遇到迷茫的时候, 不妨考虑具体的自编码器类需要什么样的encoder, decoder.
class Decoder(nn.Module):
    def __init__(self, x_dim, hidden_size, latent_size, decode_type="AE", **kwargs) -> None:
        super(Decoder, self).__init__()
        if decode_type == "AE":
            self.decoder = nn.Sequential(nn.Linear(latent_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, x_dim),)
        elif decode_type == "VAE":
            self.decoder = ...  # 实现VAE的decoder (5/100)
        elif decode_type == "CVAE":
            self.decoder = ...  # 实现CVAE的decoder (5/100)
        else:
            raise NotImplementedError

    def forward(self, zs, **otherinputs):
        # 实现decoder的decode部分, 注意对不同的decode_type的处理与对**otherinputs的解析 (10/100)
        return ...

#### 1.1.2 基于通用部件的AE实现

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, encoder, decoder, **kwargs) -> None:
        super(AutoEncoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        z = self.encoder(x)
        # 实现AE的forward过程(5/100)
        ...
        return

In [1]:
# 数据集格式
import torch
import torch.nn as nn
import torch.utils.data as td
import torchvision.datasets.mnist as mnist
import torchvision.transforms as transforms
import numpy as np
import math
import scipy.linalg as linalg
# import matplotlib as mpl
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt
def get_data(train=True, batch_size=128):
    dataset = mnist.MNIST(
        root="./data",
        train=train,
        download=True,
        transform=transforms.Compose(
            [
                #transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize([0.1307,], [0.3081,],),
            ]
        ),
    )
    for i, (x, y) in enumerate(dataset):
        print(i,x.shape,y)
        #print(y.shape)
    return td.DataLoader(dataset, batch_size=batch_size,drop_last=True,)


if __name__ == "__main__":
    data=get_data()
    print(data)

c:\Users\86152\anaconda3\envs\bert_chinese\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


9920512it [00:00, 11097266.11it/s]                             


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


32768it [00:00, 215525.94it/s]           


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw


1654784it [00:00, 3692478.63it/s]                             


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


8192it [00:00, 53387.87it/s]            
c:\Users\86152\anaconda3\envs\bert_chinese\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!
0 torch.Size([1, 28, 28]) 5
1 torch.Size([1, 28, 28]) 0
2 torch.Size([1, 28, 28]) 4
3 torch.Size([1, 28, 28]) 1
4 torch.Size([1, 28, 28]) 9
5 torch.Size([1, 28, 28]) 2
6 torch.Size([1, 28, 28]) 1
7 torch.Size([1, 28, 28]) 3
8 torch.Size([1, 28, 28]) 1
9 torch.Size([1, 28, 28]) 4
10 torch.Size([1, 28, 28]) 3
11 torch.Size([1, 28, 28]) 5
12 torch.Size([1, 28, 28]) 3
13 torch.Size([1, 28, 28]) 6
14 torch.Size([1, 28, 28]) 1
15 torch.Size([1, 28, 28]) 7
16 torch.Size([1, 28, 28]) 2
17 torch.Size([1, 28, 28]) 8
18 torch.Size([1, 28, 28]) 6
19 torch.Size([1, 28, 28]) 9
20 torch.Size([1, 28, 28]) 4
21 torch.Size([1, 28, 28]) 0
22 torch.Size([1, 28, 28]) 9
23 torch.Size([1, 28, 28]) 1
24 torch.Size([1, 28, 28]) 1
25 torch.Size([1, 28, 28]) 2
26 torch.Size([1, 28, 28]) 4
27 torch.Size([1, 28, 28]) 3
28 torch.Size([1, 28, 28]) 2
29 torch.Size([1, 28, 28]) 7
30 torch.Size([1, 28, 28]) 3
31 torch.Size([1,

KeyboardInterrupt: 